In [1]:
# Define the gpu  on the gpu machine
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=6

import pandas as pd
import json
from tqdm import tqdm
from transformers import AutoTokenizer
from datasets import load_dataset
import numpy as np
from collections import Counter
from sklearn.preprocessing import MinMaxScaler
import re

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=6


/home/tajak/Crosslingual-Genre-Bias-Analysis/crosslingual_venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Tokenize and count tokens for train_df

Code for tokenization (it is now already done):

In [18]:
# Import the train dataset
train = load_dataset("TajaKuzman/X-GENRE-multilingual-text-genre-dataset", "train")

# To open as Pandas DataFrame:
train_df = pd.DataFrame(train["train"])

display(train_df.head(2))

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

# Tokenize the train dataset
tokens_train = []
integers_train = []
token_list_train = []

for text in tqdm(train_df.text.to_list()):
	encoded_text = tokenizer(text)
	# Take all tokens_train, except the beginning (<s>) and end (</s>) token
	current_tokens_train = tokenizer.convert_ids_to_tokens(encoded_text.input_ids)[1:-1]
	tokens_train.append(current_tokens_train)
	token_list_train.extend(current_tokens_train)
	integers_train.append(encoded_text.input_ids[1:-1])

train_df["tokens_train"] = tokens_train
train_df["token_ids"] = integers_train

#print(token_list_train[:10])
#print(len(token_list_train))

# Create a list of tokens, where we take only the first 512 tokens
train_tokens_shortened = []

for i in train_df["tokens_train"].to_list():
	train_tokens_shortened.extend(i[:512])

print(len(train_tokens_shortened))

train_df.head(3)

# Save the tokenized version
train_df.to_json("datasets/tokenized_datasets/X-GENRE-train-tokenized.json")

,text,labels,dataset,language
0,"Seeking All Things Brilliant ""I want people to...",Other,CORE,English
1,Meet Orchid du Bois I first met Hayley Mowday ...,Other,CORE,English


100%|██████████| 1772/1772 [00:09<00:00, 189.74it/s]


699465


Code to count tokens:

In [6]:
# Open the tokenized df
train_df = pd.read_json("datasets/tokenized_datasets/X-GENRE-train-tokenized.json")
train_df.head(2)

,text,labels,dataset,language,tokens_train,token_ids
0,"Seeking All Things Brilliant ""I want people to...",Other,CORE,English,"[▁See, king, ▁All, ▁Things, ▁Br, illian, t, ▁""...","[6872, 6048, 3164, 119175, 13008, 162076, 18, ..."
1,Meet Orchid du Bois I first met Hayley Mowday ...,Other,CORE,English,"[▁Meet, ▁Or, ch, id, ▁du, ▁Bo, is, ▁I, ▁first,...","[72626, 3347, 206, 532, 115, 2460, 164, 87, 51..."


In [7]:
# Create a dictionary that counts all the token occurrences

# Create a list of tokens, where we take only the first 512 tokens
train_tokens_shortened = []

for i in train_df["tokens_train"].to_list():
	train_tokens_shortened.extend(i[:512])

print(len(train_tokens_shortened))

# Create a dictionary which counts the occurrences of the words

word_dict_train = Counter(train_tokens_shortened)

# Sort the dictionary alphabetically based on keys
word_dict_train = dict(sorted(word_dict_train.items()))

print(list(word_dict_train.items())[:100])
print(len(word_dict_train))

699465
[('!', 430), ('!!', 23), ('!!!', 14), ('!!!!', 6), ('!!!!!', 1), ('!!!!!!', 1), ('!!!!!!!', 1), ('!"', 14), ('!)', 10), ('!),', 1), ('!).', 2), ('"', 528), ('")', 7), ('"),', 6), ('").', 7), ('",', 56), ('".', 83), ('"...', 3), ('";', 3), ('"?', 6), ('#', 4), ('$', 8), ('%', 4), ('%)', 1), ('&', 46), ("'", 4517), ('(', 31), ('(1', 1), (')', 788), ('),', 242), (').', 297), ('):', 13), (');', 5), ('*', 19), ('**', 2), ('****', 1), ('+', 15), ('+5', 1), (',', 23447), (',«', 12), ('-', 2803), ('---', 3), ('------', 5), ('----------------', 41), ('-0', 3), ('-01', 1), ('-02', 2), ('-02-', 2), ('-03-', 4), ('-06', 1), ('-06-', 3), ('-09-', 1), ('-1', 15), ('-1)', 3), ('-10', 2), ('-10-', 1), ('-11', 6), ('-11-', 1), ('-12', 8), ('-13', 6), ('-14', 3), ('-15', 7), ('-16', 9), ('-17', 3), ('-18', 9), ('-19', 5), ('-2', 10), ('-20', 3), ('-2000', 3), ('-2005', 1), ('-2007', 8), ('-2009', 1), ('-2010', 1), ('-2011', 1), ('-2012', 1), ('-2014', 1), ('-2020', 1), ('-21', 4), ('-22', 3), ('-

In [6]:
len(set(train_tokens_shortened))

27025

The train dataset has 699.465 tokens and 27.025 unique words.

In [7]:
# See the most frequent tokens:

# Sort the dictionary by values (word counts) in descending order
sorted(word_dict_train.items(), key=lambda x: x[1], reverse=True)[:10]


[(',', 23447),
 ('.', 21407),
 ('▁', 19553),
 ('▁the', 18860),
 ('s', 14184),
 ('▁to', 10762),
 ('▁of', 9912),
 ('▁and', 9752),
 ('▁in', 9140),
 ('▁a', 8341)]

In [14]:
# Save the dictionary of tokens
with open("datasets/tokenized_datasets/X-GENRE-train-token-count.json", "w") as train_count_file:
	json.dump(word_dict_train, train_count_file)

# Tokenize and count tokens for test sets & calculate percentage overlap

Code with which I tokenized the datasets:

In [4]:
# Import the final dataset with test sets
with open("manual-annotations/multilingual-genre-annotated-test-set.json") as main_file:
	main_dict = json.load(main_file)

main_dict.keys()

dict_keys(['mt', 'el', 'tr', 'sq', 'is', 'uk', 'ca', 'mk', 'hr', 'sl'])

In [ ]:
# Code with which the test sets were tokenized

# Define an array of token overlap
token_overlap_results = {}

# Loop through the datasets and calculate token overlap
for lang in list(main_dict.keys()):
	df = pd.DataFrame(main_dict[lang]["dataset"])

	tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

	tokens = []
	integers = []
	token_list = []

	print("Tokenizing text.")

	for text in tqdm(df.text.to_list()):
		encoded_text = tokenizer(text)
		# Take all tokens, except the beginning (<s>) and end (</s>) token
		current_tokens = tokenizer.convert_ids_to_tokens(encoded_text.input_ids)[1:-1]
		# Shorten the list to 512, as tokens after that were not observed by the classifier
		current_tokens = current_tokens[:512]
		tokens.append(current_tokens)
		token_list.extend(current_tokens)
		integers.append(encoded_text.input_ids[1:-1][:512])

	df["tokens"] = tokens
	df["token_ids"] = integers

	print(token_list[:10])
	print("All tokens:")
	print(len(token_list))

	# See how many tokens overlap
	overlap_counter = 0

	print("Calculating overlap.")

	# Save tokens that overlap for further inspection
	overlap_token_list = []

	# We calculate overlap by dividing the number of tokens that are shared in train and test set by the number of all tokens in both sets
	#overlap_counter = sum(1 for element in token_list if element in train_tokens_shortened)
	for token in tqdm(token_list):
		if token in train_tokens_shortened:
			# Add to the overlap counter number of all times the token appears in both datasets
			overlap_counter += 1
			overlap_token_list.append(token)


	# Out of all tokens in test set, how many overlap?
	overlap_per = overlap_counter/len(token_list)

	print(f"Number of tokens that overlap: {overlap_counter}")
	print(f"Percentage of overlap: {overlap_per}")

	# Update the dataset in the dictionary
	main_dict[lang]["dataset"] = df.to_dict()

	# Add the list of all tokens to the dictionary
	main_dict[lang]["token_overlap"] = {"overlap_percentage":overlap_per, "token_list": token_list, "overlap_token_list":overlap_token_list}

	# Add to the results
	token_overlap_results[lang] = {"percentage": overlap_per, "overlap_list": overlap_token_list}

	# Convert tokens back to words
	#print(tokenizer.convert_tokens_to_string(tokens))

# Inspect the results
overlap_df = pd.DataFrame(token_overlap_results)

In [8]:
# Improved code to calculate token overlap - we can simply calculate which tokens do not overlap - meaning that they occur only in one of the dataset, and calculate the percentage overlap based on that
token_overlap_results = {}

# Loop through the datasets and calculate token overlap
for lang in list(main_dict.keys()):
	token_list = main_dict[lang]["token_overlap"]['token_list']
	print(token_list[:10])
	print("All tokens:")
	print(len(token_list))

	# See how many tokens do not overlap
	no_overlap_counter = 0

	print("Calculating overlap.")

	# Save tokens that overlap for further inspection
	overlap_token_list = []

	# We calculate overlap by counting how many tokens do not appear in both sets
	for token in tqdm(token_list):
		if token not in train_tokens_shortened:
			no_overlap_counter += 1
		else:
			overlap_token_list.append(token)

	# Out of all tokens in test set, how many do not overlap with train set?
	no_overlap_per = no_overlap_counter/len(token_list)

	# Calculate percentage of overlap based on that
	overlap_per = 1-no_overlap_per

	print(f"Number of tokens that overlap: {len(overlap_token_list)}")
	print(f"Number of different tokens that overlap: {len(set(overlap_token_list))}")
	print(f"Percentage of overlap: {overlap_per}")

	# Add the list of all tokens to the dictionary
	main_dict[lang]["token_overlap"]["overlap_percentage"] = overlap_per
	main_dict[lang]["token_overlap"]["overlap_token_list"] = overlap_token_list

	# Add to the results
	token_overlap_results[lang] = {"percentage": overlap_per, "overlap_list_size": len(overlap_token_list), "overlap_set_size": len(set(overlap_token_list))}


# Inspect the results
overlap_df = pd.DataFrame(token_overlap_results)

['▁Angel', 'o', '▁Che', 't', 'cuti', ',', '▁se', '▁j', 'kun', '▁qe']
All tokens:
39040
Calculating overlap.


100%|██████████| 39040/39040 [01:28<00:00, 439.17it/s] 


Number of tokens that overlap: 31899
Number of different tokens that overlap: 3586
Percentage of overlap: 0.8170850409836066
['▁Ενημέρωση', '▁του', '▁Pegasus', '▁Esti', 'asi', '▁με', '▁τις', '▁εισ', 'ερ', 'χ']
All tokens:
31240
Calculating overlap.


100%|██████████| 31240/31240 [03:41<00:00, 141.06it/s]


Number of tokens that overlap: 5043
Number of different tokens that overlap: 822
Percentage of overlap: 0.1614276568501921
['▁A', 'Ö', 'L', '▁Der', 's', '▁Seçim', 'i', '▁ve', '▁Sınav', '▁Giriş']
All tokens:
29578
Calculating overlap.


...

Number of tokens that overlap: 4035
Number of different tokens that overlap: 656
Percentage of overlap: 0.14598936285683273
['▁O', '▁proizvod', 'u', '▁Color', '▁Trans', 'former', ',', '▁za', '▁pamet', 'no']
All tokens:
26546
Calculating overlap.


100%|██████████| 26546/26546 [01:08<00:00, 389.67it/s]


Number of tokens that overlap: 21808
Number of different tokens that overlap: 4383
Percentage of overlap: 0.8215173660815189
['▁Kita', 'jsko', '▁mesto', '▁duhov', '▁V', '▁Notranj', 'i', '▁Mongol', 'iji', '▁raste']
All tokens:
26292
Calculating overlap.


100%|██████████| 26292/26292 [00:49<00:00, 527.18it/s]

Number of tokens that overlap: 25616
Number of different tokens that overlap: 5281
Percentage of overlap: 0.9742887570363609


In [9]:
overlap_df = pd.DataFrame(token_overlap_results).transpose()
overlap_df

,percentage,overlap_list_size,overlap_set_size
mt,0.817085,31899.0,3586.0
el,0.161428,5043.0,822.0
tr,0.521502,15425.0,2451.0
sq,0.605775,16216.0,2748.0
is,0.517575,15343.0,2122.0
uk,0.156675,4963.0,411.0
ca,0.744881,20517.0,2896.0
mk,0.145989,4035.0,656.0
hr,0.821517,21808.0,4383.0
sl,0.974289,25616.0,5281.0


In [10]:
overlap_df.sort_values(by="percentage", ascending=False)

,percentage,overlap_list_size,overlap_set_size
sl,0.974289,25616.0,5281.0
hr,0.821517,21808.0,4383.0
mt,0.817085,31899.0,3586.0
ca,0.744881,20517.0,2896.0
sq,0.605775,16216.0,2748.0
tr,0.521502,15425.0,2451.0
is,0.517575,15343.0,2122.0
el,0.161428,5043.0,822.0
uk,0.156675,4963.0,411.0
mk,0.145989,4035.0,656.0


In [11]:
# Save the extended json dict
with open("manual-annotations/multilingual-genre-annotated-test-set.json", "w") as file:
	json.dump(main_dict, file)

Add token counts information

In [23]:
# Import the final dataset with test sets
with open("manual-annotations/multilingual-genre-annotated-test-set.json") as main_file:
	main_dict = json.load(main_file)

main_dict.keys()

dict_keys(['mt', 'el', 'tr', 'sq', 'is', 'uk', 'ca', 'mk', 'hr', 'sl'])

In [48]:
token_number = {}
type_number = {}

for lang in list(main_dict.keys()):
	#print(f"Creating token dict for {lang}")
	current_token_list = main_dict[lang]["token_overlap"]["token_list"]

	#print(f"No of tokens: {len(current_token_list)}")

	# Create a dictionary which counts the occurrences of the tokens

	#word_dict_test = Counter(current_token_list)

	# Sort the dictionary alphabetically based on keys
	#word_dict_test = dict(sorted(word_dict_test.items()))

	# Add information on no. of tokens and words to a dict
	token_number[lang] = len(current_token_list)
	#type_number[lang] = len(word_dict_test)
	type_number[lang] = len(set(current_token_list))

	#print(list(word_dict_test.items())[:100])
	#print(f"No of unique tokens: {len(word_dict_test)}")

	# Add the count of tokens to the main dictionary
	#main_dict[lang]["token_overlap"]["token_count"] = word_dict_test

# Create a dataframe for statistics
token_df = pd.DataFrame({"tokens": token_number, "types": type_number})
print(token_df.to_markdown())

|    |   tokens |   types |
|:---|---------:|--------:|
| mt |    39040 |    4787 |
| el |    31240 |    4751 |
| tr |    29578 |    6272 |
| sq |    26769 |    4891 |
| is |    29644 |    4615 |
| uk |    31677 |    6507 |
| ca |    27544 |    5314 |
| mk |    27639 |    5468 |
| hr |    26546 |    6222 |
| sl |    26292 |    5763 |


In [59]:
# Merge token_df and overlap_df
merged = pd.concat([overlap_df, token_df], axis=1)
merged

# Reorder columns

merged = merged[["percentage", "tokens", "overlap_list_size", "types","overlap_set_size"]]

# Rename columns

merged.columns = ["overlap_percentage", "all_tokens", "overlapping_tokens", "all_types", "overlapping_types"]

merged

,overlap_percentage,all_tokens,overlapping_tokens,all_types,overlapping_types
mt,0.817085,39040,31899.0,4787,3586.0
el,0.161428,31240,5043.0,4751,822.0
tr,0.521502,29578,15425.0,6272,2451.0
sq,0.605775,26769,16216.0,4891,2748.0
is,0.517575,29644,15343.0,4615,2122.0
uk,0.156675,31677,4963.0,6507,411.0
ca,0.744881,27544,20517.0,5314,2896.0
mk,0.145989,27639,4035.0,5468,656.0
hr,0.821517,26546,21808.0,6222,4383.0
sl,0.974289,26292,25616.0,5763,5281.0


In [61]:
print(merged.to_markdown())

|    |   overlap_percentage |   all_tokens |   overlapping_tokens |   all_types |   overlapping_types |
|:---|---------------------:|-------------:|---------------------:|------------:|--------------------:|
| mt |             0.817085 |        39040 |                31899 |        4787 |                3586 |
| el |             0.161428 |        31240 |                 5043 |        4751 |                 822 |
| tr |             0.521502 |        29578 |                15425 |        6272 |                2451 |
| sq |             0.605775 |        26769 |                16216 |        4891 |                2748 |
| is |             0.517575 |        29644 |                15343 |        4615 |                2122 |
| uk |             0.156675 |        31677 |                 4963 |        6507 |                 411 |
| ca |             0.744881 |        27544 |                20517 |        5314 |                2896 |
| mk |             0.145989 |        27639 |                 403

In [35]:
# Let's see the most frequent tokens
most_frequent = {}
# Sort the dictionary by values (word counts) in descending order
for lang in list(main_dict.keys()):
	most_frequent[lang] = (sorted(main_dict[lang]['token_overlap']['token_count'].items(), key=lambda x: x[1], reverse=True)[:10])

print(pd.DataFrame({"most_frequent_type": most_frequent}).to_markdown())

|    | most_frequent_type                                                                                                                       |
|:---|:-----------------------------------------------------------------------------------------------------------------------------------------|
| ca | [(',', 1080), ('▁de', 1014), ('.', 675), ('s', 648), ('▁i', 564), ('▁la', 559), ('▁a', 529), ('▁que', 438), ("'", 360), ('’', 334)]      |
| el | [('▁', 1017), ('.', 801), (',', 782), ('▁και', 553), ('ς', 525), ('▁να', 351), ('▁το', 321), ('▁του', 292), ('▁την', 268), ('▁με', 264)] |
| hr | [(',', 878), ('.', 766), ('▁i', 546), ('▁u', 430), ('a', 413), ('▁je', 350), ('▁na', 282), ('▁za', 253), ('▁se', 239), ('e', 219)]       |
| is | [('.', 1017), ('▁og', 628), ('▁að', 613), (',', 600), ('▁', 528), ('▁í', 434), ('▁á', 388), ('▁er', 357), ('s', 326), ('▁sem', 279)]     |
| mk | [(',', 1021), ('▁на', 983), ('.', 738), ('▁и', 619), ('▁за', 475), ('▁да', 378), ('▁во', 376), ('▁се', 365), ('▁', 34

In [36]:
# Save the main dict
with open("manual-annotations/multilingual-genre-annotated-test-set.json", "w") as file:
	json.dump(main_dict, file)

### Cosine similarity

Create a dictionary with all tokens from the train set, iterate through the tokens of each test set and count how many times each ocurrs. Calculate cosine similarity.

Do this on label level as well to see whether this explains good performance on some of the labels for Maltese.

Cosine similarity is a metric used to measure the similarity of two vectors. Specifically, it measures the similarity in the direction or orientation of the vectors ignoring differences in their magnitude or scale. Both vectors need to be part of the same inner product space, meaning they must produce a scalar through inner product multiplication. The similarity of two vectors is measured by the cosine of the angle between them. The similarity can take values between -1 and +1. Smaller angles between vectors produce larger cosine values, indicating greater cosine similarity. 

Cosine similarity ignores 0-0 matches. Counting 0-0 matches in sparse data would inflate similarity scores. Another commonly used metric that ignores 0-0 matches is Jaccard Similarity.

In [2]:
def cosine_similarity(x, y):
    
    # Ensure length of x and y are the same
    if len(x) != len(y) :
        return None
    
    # Compute the dot product between x and y
    dot_product = np.dot(x, y)
    
    # Compute the L2 norms (magnitudes) of x and y
    magnitude_x = np.sqrt(np.sum(x**2)) 
    magnitude_y = np.sqrt(np.sum(y**2))
    
    # Compute the cosine similarity
    cosine_similarity = dot_product / (magnitude_x * magnitude_y)
    
    return cosine_similarity

In [3]:
# Import train token count
# Save the dictionary of tokens
with open("datasets/tokenized_datasets/X-GENRE-train-token-count.json", "r") as train_count_file:
	train_count = json.load(train_count_file)

list(train_count.items())[:5]

[('!', 430), ('!!', 23), ('!!!', 14), ('!!!!', 6), ('!!!!!', 1)]

In [4]:
# Import the main dict for test sets
with open("manual-annotations/multilingual-genre-annotated-test-set.json", "r") as file:
	main_dict = json.load(file)

list(main_dict["sl"]["token_overlap"]["token_count"].items())[:10]

[('!', 30),
 ('!!', 1),
 ('!"', 1),
 ('!),', 1),
 ('"', 11),
 ('".', 2),
 ('"...', 1),
 ('&', 8),
 ("'", 3),
 ("''", 1)]

In [6]:
cosine_sim = {}
vector_size = {}

for lang in list(main_dict.keys()):
	print(f"Processing {lang}")
	# Get token count for current lang
	current_lang_count = main_dict[lang]["token_overlap"]["token_count"]

	print(f"Number of token types for {lang}: {len(list(current_lang_count.keys()))}")

	# For each test set, create a vector of token counts. Take only tokens that are present in train_df.

	# First, create a list of tokens that are present in either one or the other list
	intersection_keys = list(train_count.keys())

	# Then create a dictionary for 1) train df and 2) test df with counts of token types that occur in train_df
	train_intersect_dict = train_count
	test_intersect_dict = {}

	for i in intersection_keys:
		try:
			test_intersect_dict[i] = current_lang_count[i]
		except:
			test_intersect_dict[i] = 0


	# Create a df with intersecting keys
	intersect_df = pd.DataFrame({"train": train_intersect_dict, "test": test_intersect_dict})

	display(intersect_df.head(10))

	# Calculate cosine similarity
	current_cosine_sim = cosine_similarity(np.array(intersect_df["train"].to_list()), np.array(intersect_df["test"].to_list()))

	print(f"Cosine similarity for {lang}: {current_cosine_sim}")

	# Add to the main dictionary
	main_dict[lang]["token_overlap"]["cosine_similarity"] = current_cosine_sim
	main_dict[lang]["token_overlap"]["intersection_df"] = intersect_df.to_dict()
	main_dict[lang]["token_overlap"]["intersection_vector_size"] = len(intersection_keys)

	# Add to a dict of results
	cosine_sim[lang] = current_cosine_sim
	vector_size[lang] = len(intersection_keys)

Processing mt
Number of token types for mt: 4787


,train,test
!,430,9
!!,23,0
!!!,14,0
!!!!,6,0
!!!!!,1,0
!!!!!!,1,0
!!!!!!!,1,0
"!""",14,0
!),10,0
"!),",1,0


Cosine similarity for mt: 0.49192396040790515
Processing el
Number of token types for el: 4751


,train,test
!,430,23
!!,23,2
!!!,14,5
!!!!,6,0
!!!!!,1,0
!!!!!!,1,1
!!!!!!!,1,0
"!""",14,0
!),10,0
"!),",1,0


Cosine similarity for el: 0.7069651767644073
Processing tr
Number of token types for tr: 6272


...

Cosine similarity for mk: 0.689022808410823
Processing hr
Number of token types for hr: 6222


,train,test
!,430,33
!!,23,1
!!!,14,3
!!!!,6,0
!!!!!,1,0
!!!!!!,1,0
!!!!!!!,1,0
"!""",14,0
!),10,0
"!),",1,0


Cosine similarity for hr: 0.5699962229590452
Processing sl
Number of token types for sl: 5763


,train,test
!,430,30
!!,23,1
!!!,14,0
!!!!,6,0
!!!!!,1,0
!!!!!!,1,0
!!!!!!!,1,0
"!""",14,1
!),10,0
"!),",1,1


Cosine similarity for sl: 0.6335818041038892


In [34]:
# Create a list of tokens in train_df with only tokens that have at least one letter character - so, we remove tokens that consist only of punctuation or numbers
train_count_lexical_list = [item for item in list(train_count.keys()) if any(char.isalpha() for char in item)]

# Additionally, we keep only tokens that are longer than 1 character

train_count_lexical = {x:train_count[x] for x in train_count_lexical_list if len(x.replace("▁", "")) > 1}



list(train_count_lexical.items())[:4]

[('AA', 10), ('AAN', 1), ('AAR', 1), ('AB', 1)]

In [35]:
len(list(train_count_lexical.keys()))

25225

In [36]:
# experiment with removing punctuation, white spaces and numbers from the token list

cosine_sim = {}
vector_size = {}

for lang in list(main_dict.keys()):
	print(f"Processing {lang}")
	# Get token count for current lang
	current_lang_count = main_dict[lang]["token_overlap"]["token_count"]

	print(f"Number of token types for {lang}: {len(list(current_lang_count.keys()))}")

	# For each test set, create a vector of token counts. Take only tokens that are present in train_df.

	# First, create a list of tokens that are present in either one or the other list

	intersection_keys = list(train_count_lexical.keys())

	# Then create a dictionary for 1) train df and 2) test df with counts of token types that occur in train_df
	train_intersect_dict = train_count_lexical
	test_intersect_dict = {}

	for i in intersection_keys:
		try:
			test_intersect_dict[i] = current_lang_count[i]
		except:
			test_intersect_dict[i] = 0


	# Create a df with intersecting keys
	intersect_df = pd.DataFrame({"train": train_intersect_dict, "test": test_intersect_dict})

	display(intersect_df.head(10))

	# Calculate cosine similarity
	current_cosine_sim = cosine_similarity(np.array(intersect_df["train"].to_list()), np.array(intersect_df["test"].to_list()))

	print(f"Cosine similarity for {lang}: {current_cosine_sim}")

	# Add to the main dictionary
	main_dict[lang]["token_overlap"]["cosine_similarity"] = current_cosine_sim
	main_dict[lang]["token_overlap"]["intersection_df"] = intersect_df.to_dict()
	main_dict[lang]["token_overlap"]["intersection_vector_size"] = len(intersection_keys)

	# Add to a dict of results
	cosine_sim[lang] = current_cosine_sim
	vector_size[lang] = len(intersection_keys)

Processing mt
Number of token types for mt: 4787


,train,test
AA,10,0
AAN,1,0
AAR,1,0
AB,1,1
ABILI,1,0
ABLE,6,0
AC,3,0
ACE,8,0
ACH,3,0
ACI,2,0


Cosine similarity for mt: 0.2842150848173066
Processing el
Number of token types for el: 4751


,train,test
AA,10,0
AAN,1,0
AAR,1,0
AB,1,0
ABILI,1,0
ABLE,6,0
AC,3,0
ACE,8,0
ACH,3,0
ACI,2,0


Cosine similarity for el: 0.12768966885809033
Processing tr
Number of token types for tr: 6272


...

Cosine similarity for mk: 0.22877625063792426
Processing hr
Number of token types for hr: 6222


,train,test
AA,10,0
AAN,1,0
AAR,1,0
AB,1,0
ABILI,1,0
ABLE,6,0
AC,3,0
ACE,8,0
ACH,3,0
ACI,2,0


Cosine similarity for hr: 0.2076741403073573
Processing sl
Number of token types for sl: 5763


,train,test
AA,10,0
AAN,1,0
AAR,1,0
AB,1,0
ABILI,1,0
ABLE,6,0
AC,3,0
ACE,8,0
ACH,3,1
ACI,2,0


Cosine similarity for sl: 0.331293659491296


In [9]:
# Experiment with normalizing the values

In [13]:
# Experiment what happens if we normalize the values using min max
# Normalize the values using the min-max normalization
# Min-max is a scaling technique where values are rescaled and shifted so that they range between 0 and 1 or between -1 and 1.

def normalize(list):
	data = np.array(list).reshape(-1, 1)
	scaler = MinMaxScaler()
	scaler.fit(data)
	norm_list = scaler.transform(data)

	return norm_list

In [14]:
df = pd.DataFrame(main_dict["uk"]["token_overlap"]["intersection_df"])
df.head(4)

,train,test
!,430,41
!!,23,1
!!!,14,6
!!!!,6,0


In [ ]:
cosine_similarity(np.array(df["train"].to_list()), np.array(df["test"].to_list()))

In [ ]:
cosine_similarity(np.array(normalize(df["train"].to_list())), np.array(normalize(df["test"].to_list())))

In [16]:
normalize(df["test"].to_list())

array([[0.03032544],
       [0.00073964],
       [0.00443787],
       ...,
       [0.        ],
       [0.        ],
       [0.        ]])

In [8]:
# Show results
cosine_sim_df = pd.DataFrame({"cosine_similarity": cosine_sim, "vector_size": vector_size}).sort_values(by="cosine_similarity", ascending=False)
print(cosine_sim_df.to_markdown())

|    |   cosine_similarity |   vector_size |
|:---|--------------------:|--------------:|
| el |            0.706965 |         27025 |
| uk |            0.690846 |         27025 |
| mk |            0.689023 |         27025 |
| sl |            0.633582 |         27025 |
| tr |            0.615202 |         27025 |
| is |            0.610181 |         27025 |
| hr |            0.569996 |         27025 |
| sq |            0.545689 |         27025 |
| ca |            0.529031 |         27025 |
| mt |            0.491924 |         27025 |


In [11]:
overlap_per = {}
for lang in list(main_dict.keys()):
	overlap_per[lang] = main_dict[lang]["token_overlap"]["overlap_percentage"]

In [12]:
# Show results
cosine_sim_df = pd.DataFrame({"cosine_similarity": cosine_sim, "vector_size": vector_size, "overlap_percentage": overlap_per}).sort_values(by="cosine_similarity", ascending=False)
print(cosine_sim_df.to_markdown())

|    |   cosine_similarity |   vector_size |   overlap_percentage |
|:---|--------------------:|--------------:|---------------------:|
| el |            0.706965 |         27025 |             0.161428 |
| uk |            0.690846 |         27025 |             0.156675 |
| mk |            0.689023 |         27025 |             0.145989 |
| sl |            0.633582 |         27025 |             0.974289 |
| tr |            0.615202 |         27025 |             0.521502 |
| is |            0.610181 |         27025 |             0.517575 |
| hr |            0.569996 |         27025 |             0.821517 |
| sq |            0.545689 |         27025 |             0.605775 |
| ca |            0.529031 |         27025 |             0.744881 |
| mt |            0.491924 |         27025 |             0.817085 |


# Compare token overlap on label level 

### Create label-level token counts for train dataset

In [3]:
# Open the tokenized train df
train_df = pd.read_json("datasets/tokenized_datasets/X-GENRE-train-tokenized.json")
train_df.head(2)

,text,labels,dataset,language,tokens_train,token_ids
0,"Seeking All Things Brilliant ""I want people to...",Other,CORE,English,"[▁See, king, ▁All, ▁Things, ▁Br, illian, t, ▁""...","[6872, 6048, 3164, 119175, 13008, 162076, 18, ..."
1,Meet Orchid du Bois I first met Hayley Mowday ...,Other,CORE,English,"[▁Meet, ▁Or, ch, id, ▁du, ▁Bo, is, ▁I, ▁first,...","[72626, 3347, 206, 532, 115, 2460, 164, 87, 51..."


In [4]:
train_df.labels.unique()

array(['Other', 'Information/Explanation', 'News', 'Instruction',
       'Opinion/Argumentation', 'Forum', 'Prose/Lyrical', 'Legal',
       'Promotion'], dtype=object)

In [ ]:
# Separate the train df into label-based dfs

# Create lists of tokens that are label based
label_tokens = {}
token_count = {}
type_count = {}

for label in ['Information/Explanation', 'News', 'Instruction','Opinion/Argumentation', 'Forum', 'Prose/Lyrical', 'Legal','Promotion']:
	print(f"Processing {label}")

	label_df = train_df[train_df["labels"] == label]
	display(label_df.head(3))

	# Create a list of tokens, where we take only the first 512 tokens
	train_tokens_shortened = []

	for i in label_df["tokens_train"].to_list():
		train_tokens_shortened.extend(i[:512])

	print(f"Number of all tokens: {len(train_tokens_shortened)}")

	# Add to dictionaries
	label_tokens[label] = train_tokens_shortened
	token_count[label] = len(train_tokens_shortened)
	type_count[label] = len(set(train_tokens_shortened))

In [8]:
# Calculate numbers for each label
label_results_train = pd.DataFrame({"token_count": token_count, "type_count": type_count})

print(label_results_train.to_markdown())

|                         |   token_count |   type_count |
|:------------------------|--------------:|-------------:|
| Information/Explanation |        124130 |        14678 |
| News                    |        136557 |        15319 |
| Instruction             |         83750 |         8929 |
| Opinion/Argumentation   |        103141 |        13088 |
| Forum                   |         58900 |         8555 |
| Prose/Lyrical           |         46860 |         5990 |
| Legal                   |         28496 |         4425 |
| Promotion               |         88626 |        12548 |


In [9]:
# Save the label token count
with open("datasets/tokenized_datasets/X-GENRE-train-label-token-count.json", "w") as train_label_count_file:
	json.dump(label_tokens, train_label_count_file)

### Create label-level token counts for test sets

In [12]:
# Import the final dataset with test sets
with open("manual-annotations/multilingual-genre-annotated-test-set.json") as main_file:
	main_dict = json.load(main_file)

main_dict["uk"].keys()

dict_keys(['accuracy', 'micro_f1', 'macro_f1', 'dataset', 'token_overlap'])

In [13]:
pd.DataFrame(main_dict["sl"]["dataset"]).head(2)

,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids
1,CLASSLA-web.sl.1087171,Opinion/Argumentation,Kitajsko mesto duhov\nV Notranji Mongoliji ras...,The Chinese ghost city in Inner Mongolia is gr...,"CLASSLA-web.sl.1087171', 'domain': 'mladina.si'}",News,"[▁Kita, jsko, ▁mesto, ▁duhov, ▁V, ▁Notranj, i,...","[12992, 30878, 22041, 87909, 310, 240213, 14, ..."
2,CLASSLA-web.sl.1215246,Instruction,"Krompir skuhamo, olupimo in narežemo na tanke ...","Cook the potatoes, peel and cut into thin ring...","CLASSLA-web.sl.1215246', 'domain': 'emorje.com'}",Instruction,"[▁Krom, pir, ▁s, kuha, mo, ,, ▁olup, imo, ▁in,...","[128458, 21738, 91, 60863, 432, 4, 37663, 2414..."


In [14]:
# Do the same as with the dataset, but on every language
lang_results = {}

for lang in list(main_dict.keys()):
	print(f"Processing {lang}")

	label_token_dict = {}

	# Current df
	df = pd.DataFrame(main_dict[lang]["dataset"])
	display(df.head(2))

	for label in ['Information/Explanation', 'News', 'Instruction','Opinion/Argumentation', 'Forum', 'Prose/Lyrical', 'Legal','Promotion']:
		print(f"Processing {label}")

		label_df = df[df["y_true"] == label]

		# Create a list of tokens
		token_list = []

		for i in label_df["tokens"].to_list():
			token_list.extend(i)

		# Add to dictionary
		label_token_dict[label] = token_list

	# Add to main dict
	main_dict[lang]["token_overlap"]["label_level_token_lists"] = label_token_dict


Processing mt


,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids
0,macocu.mt.402244,News,"Angelo Chetcuti, se jkun qed jieħu post Bjorn ...","Angelo Chetcuti, will be replacing Bjorn Vassa...",{'text_id': 'macocu.mt.402244'},News,"[▁Angel, o, ▁Che, t, cuti, ,, ▁se, ▁j, kun, ▁q...","[26902, 31, 5024, 18, 64969, 4, 40, 1647, 6262..."
1,macocu.mt.377203,Prose/Lyrical,Poltergeist jirreferi għal fenomeni oħra tal-m...,"Poltergeist refers to other woman's phenomena,...",{'text_id': 'macocu.mt.377203'},Opinion/Argumentation,"[▁Pol, ter, geist, ▁jir, re, feri, ▁g, ħ, al, ...","[9017, 720, 178490, 52826, 107, 26926, 706, 24..."


Processing Information/Explanation
Processing News
Processing Instruction
Processing Opinion/Argumentation
Processing Forum
Processing Prose/Lyrical
Processing Legal
Processing Promotion
Processing el


,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids
0,macocu.el.1525713,Instruction,Ενημέρωση του Pegasus Estiasi με τις εισερχόμε...,Update Pegasus Estiasi with Incoming Calls\n\n...,{'text_id': 'macocu.el.1525713'},Instruction,"[▁Ενημέρωση, ▁του, ▁Pegasus, ▁Esti, asi, ▁με, ...","[236422, 385, 241060, 60271, 1544, 558, 1713, ..."
1,macocu.el.3525724,Forum,Η τιμή της έκδοσης 8GB/ 128GB είναι 1.299 ευρώ...,"The price of 8GB/ 128GB is € 1,299, of the 12G...",{'text_id': 'macocu.el.3525724'},Forum,"[▁Η, ▁τιμή, ▁της, ▁έκδοση, ς, ▁8, GB, /, ▁128,...","[1700, 77118, 463, 110873, 235, 382, 8359, 64,..."


Processing Information/Explanation
Processing News
Processing Instruction
Processing Opinion/Argumentation
Processing Forum
Processing Prose/Lyrical
Processing Legal
Processing Promotion
Processing tr


...

Processing Information/Explanation
Processing News
Processing Instruction
Processing Opinion/Argumentation
Processing Forum
Processing Prose/Lyrical
Processing Legal
Processing Promotion
Processing hr


,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids
0,CLASSLA-web.hr.1033815,Promotion,"O proizvodu\nColor Transformer, za pametno i j...","About the Color Transformer product, for smart...","CLASSLA-web.hr.1033815', 'domain': 'hairshop.hr'}",Promotion,"[▁O, ▁proizvod, u, ▁Color, ▁Trans, former, ,, ...","[180, 43170, 34, 51193, 11062, 82772, 4, 80, 6..."
2,CLASSLA-web.hr.1119579,Promotion,Sunčano selo / Sunny village\nNa obroncima Bil...,Sunshine / Sunny Village on the slopes of Bilo...,"CLASSLA-web.hr.1119579', 'domain': 'vikendi.com'}",Promotion,"[▁Sun, čan, o, ▁se, lo, ▁/, ▁Sunny, ▁village, ...","[7550, 17129, 31, 40, 365, 248, 151197, 54427,..."


Processing Information/Explanation
Processing News
Processing Instruction
Processing Opinion/Argumentation
Processing Forum
Processing Prose/Lyrical
Processing Legal
Processing Promotion
Processing sl


,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids
1,CLASSLA-web.sl.1087171,Opinion/Argumentation,Kitajsko mesto duhov\nV Notranji Mongoliji ras...,The Chinese ghost city in Inner Mongolia is gr...,"CLASSLA-web.sl.1087171', 'domain': 'mladina.si'}",News,"[▁Kita, jsko, ▁mesto, ▁duhov, ▁V, ▁Notranj, i,...","[12992, 30878, 22041, 87909, 310, 240213, 14, ..."
2,CLASSLA-web.sl.1215246,Instruction,"Krompir skuhamo, olupimo in narežemo na tanke ...","Cook the potatoes, peel and cut into thin ring...","CLASSLA-web.sl.1215246', 'domain': 'emorje.com'}",Instruction,"[▁Krom, pir, ▁s, kuha, mo, ,, ▁olup, imo, ▁in,...","[128458, 21738, 91, 60863, 432, 4, 37663, 2414..."


Processing Information/Explanation
Processing News
Processing Instruction
Processing Opinion/Argumentation
Processing Forum
Processing Prose/Lyrical
Processing Legal
Processing Promotion


In [15]:
# Save the extended json dict
with open("manual-annotations/multilingual-genre-annotated-test-set.json", "w") as file:
	json.dump(main_dict, file)

## Calculate overlap on label level

In [19]:
# Open label-based train token count
with open("datasets/tokenized_datasets/X-GENRE-train-label-token-count.json", "r") as train_label_count_file:
	label_token_count_train = json.load(train_label_count_file)

label_token_count_train.keys()

dict_keys(['Information/Explanation', 'News', 'Instruction', 'Opinion/Argumentation', 'Forum', 'Prose/Lyrical', 'Legal', 'Promotion'])

In [ ]:
# Import the main dict for test sets
with open("manual-annotations/multilingual-genre-annotated-test-set.json", "r") as file:
	main_dict = json.load(file)

main_dict["sl"]["token_overlap"]["label_level_token_lists"]

In [18]:
# Improved code to calculate token overlap - we can simply calculate which tokens do not overlap - meaning that they occur only in one of the dataset, and calculate the percentage overlap based on that
token_overlap_label_results = {}
results = {}

# Loop through the datasets and labels and calculate token overlap
for lang in list(main_dict.keys()):
	print(lang)
	# Create a dict for labels
	label_overlap = {}
	label_overlap_tokens = {}

	# loop through labels
	for label in ['Information/Explanation', 'News', 'Instruction','Opinion/Argumentation', 'Forum', 'Prose/Lyrical', 'Legal','Promotion']:
		print(label)
		token_list_test = main_dict[lang]["token_overlap"]["label_level_token_lists"][label]

		token_list_train = label_token_count_train[label]

		# See how many tokens do not overlap
		no_overlap_counter = 0

		# Save tokens that overlap for further inspection
		overlap_token_list = []

		# We calculate overlap by counting how many tokens do not appear in both sets
		for token in tqdm(token_list_test):
			if token not in token_list_train:
				no_overlap_counter += 1
			else:
				overlap_token_list.append(token)

		# Out of all tokens in test set, how many do not overlap with train set?
		no_overlap_per = no_overlap_counter/len(token_list_test)

		# Calculate percentage of overlap based on that
		overlap_per = 1-no_overlap_per

		print(f"Percentage of overlap: {overlap_per}")

		label_overlap[label] = overlap_per
		label_overlap_tokens[label] = overlap_token_list

		# Add to the results
		results[f"{lang}-{label}"] = overlap_per

	# Add the list of all tokens to the dictionary
	main_dict[lang]["token_overlap"]["label_overlap_percentage"] = label_overlap
	main_dict[lang]["token_overlap"]["label_overlap_token_list"] = label_overlap_tokens

mt
Information/Explanation


100%|██████████| 6656/6656 [00:03<00:00, 2037.47it/s]


Percentage of overlap: 0.7145432692307692
News


100%|██████████| 6978/6978 [00:03<00:00, 2046.50it/s]


Percentage of overlap: 0.7123817712811693
Instruction


...

Percentage of overlap: 0.6995531110347198
Legal


100%|██████████| 3446/3446 [00:00<00:00, 11590.15it/s]


Percentage of overlap: 0.7556587347649448
Promotion


100%|██████████| 3203/3203 [00:00<00:00, 6488.16it/s]

Percentage of overlap: 0.8741804558226662


In [21]:
results

{'mt-Information/Explanation': 0.7145432692307692,
 'mt-News': 0.7123817712811693,
 'mt-Instruction': 0.6194013781223083,
 'mt-Opinion/Argumentation': 0.68212890625,
 'mt-Forum': 0.583984375,
 'mt-Prose/Lyrical': 0.591796875,
 'mt-Legal': 0.6763671875,
 'mt-Promotion': 0.6578768288533514,
 'el-Information/Explanation': 0.13908701854493577,
 'el-News': 0.1162524992859183,
 'el-Instruction': 0.16122517670817904,
 'el-Opinion/Argumentation': 0.11699661137863382,
 'el-Forum': 0.17033287911232242,
 'el-Prose/Lyrical': 0.14589136490250698,
 'el-Legal': 0.1395486935866983,
 'el-Promotion': 0.15900735294117652,
 'tr-Information/Explanation': 0.47924858016601135,
 'tr-News': 0.44105326152004787,
 'tr-Instruction': 0.38281057051123735,
 'tr-Opinion/Argumentation': 0.4332798859789774,
 'tr-Forum': 0.44330537433985706,
 'tr-Prose/Lyrical': 0.41339001062699254,
 'tr-Legal': 0.3146586891487626,
 'tr-Promotion': 0.39420689655172414,
 'sq-Information/Explanation': 0.4844110854503464,
 'sq-News': 0.465

In [22]:
# Inspect the results
overlap_df = pd.DataFrame({"label": list(results.keys()), "overlap": list(results.values())})
overlap_df

,label,overlap
0,mt-Information/Explanation,0.714543
1,mt-News,0.712382
2,mt-Instruction,0.619401
3,mt-Opinion/Argumentation,0.682129
4,mt-Forum,0.583984
...,...,...
75,sl-Opinion/Argumentation,0.888709
76,sl-Forum,0.801008
77,sl-Prose/Lyrical,0.699553
78,sl-Legal,0.755659


In [35]:
# Save the label-level-overlap
with open("datasets/label-level-token-overlap.csv", "w") as file:
	overlap_df.to_csv(file)

In [23]:
# Save the extended json dict
with open("manual-annotations/multilingual-genre-annotated-test-set.json", "w") as file:
	json.dump(main_dict, file)

# Calculate token overlap with transliterated texts

For languages which are in non-Latin script, let's experiment also with transliteration to see what is the effect on the results.

## Prepare transliterated and normalized versions

In [10]:
# Import the final dataset with test sets
with open("manual-annotations/multilingual-genre-annotated-test-set.json") as main_file:
	main_dict = json.load(main_file)

main_dict.keys()

dict_keys(['mt', 'el', 'tr', 'sq', 'is', 'uk', 'ca', 'mk', 'hr', 'sl'])

In [25]:
main_dict["sl"]["token_overlap"].keys()

dict_keys(['overlap_percentage', 'token_list', 'overlap_token_list', 'token_count', 'cosine_similarity', 'intersection_df', 'intersection_vector_size', 'label_level_token_lists'])

In [21]:
# Transliterate the Cyrillic scripts, using the cyrtranslit library: https://github.com/opendatakosovo/cyrillic-transliteration

In [26]:
def transliterate(text, lang):
	import cyrtranslit
	transl_dict = {"uk": "ua", "bg": "bg", "cnr": "me", "mk": "mk"}
	if lang == "sr":
		transl_text = cyrtranslit.to_latin(text)
	else:
		transl_text = cyrtranslit.to_latin(text, transl_dict[lang])
	return transl_text

for lang in list(main_dict.keys()):
	if lang in ["uk", "mk"]:
		# Open the df
		df = pd.DataFrame(main_dict[lang]["dataset"])
		display(df.head(2))
		# Transliterate and save text to new column
		df["text_latin"] = [transliterate(x, lang) for x in df["text"].to_list()]
		display(df.head(2))

		# Save the df
		main_dict[lang]["dataset"] = df.to_dict()

,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids
0,macocu.uk.419381,Instruction,Нестандартний підхід для виготовлення Акваріум...,A non -standard approach for making aquarium.O...,{'text_id': 'macocu.uk.419381'},Instruction,"[▁Не, стандарт, ний, ▁підхід, ▁для, ▁виготовле...","[1087, 159257, 1394, 205827, 518, 166156, 1307..."
1,macocu.uk.16993168,Prose/Lyrical,МУЧЕНИКИ БУЧА-ІРПІНЬ \n\nНе снилось полянам й ...,The martyrs of Bucha-Irpin\n\nThe glades and t...,{'text_id': 'macocu.uk.16993168'},Prose/Lyrical,"[▁М, УЧ, ЕНИ, КИ, ▁, БУ, ЧА, -, ІР, П, ІН, Ь, ...","[1435, 87706, 78591, 38682, 6, 39932, 75333, 9..."


,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids,text_latin
0,macocu.uk.419381,Instruction,Нестандартний підхід для виготовлення Акваріум...,A non -standard approach for making aquarium.O...,{'text_id': 'macocu.uk.419381'},Instruction,"[▁Не, стандарт, ний, ▁підхід, ▁для, ▁виготовле...","[1087, 159257, 1394, 205827, 518, 166156, 1307...",Nestandartnyj pidxid dlja vyhotovlennja Akvari...
1,macocu.uk.16993168,Prose/Lyrical,МУЧЕНИКИ БУЧА-ІРПІНЬ \n\nНе снилось полянам й ...,The martyrs of Bucha-Irpin\n\nThe glades and t...,{'text_id': 'macocu.uk.16993168'},Prose/Lyrical,"[▁М, УЧ, ЕНИ, КИ, ▁, БУ, ЧА, -, ІР, П, ІН, Ь, ...","[1435, 87706, 78591, 38682, 6, 39932, 75333, 9...",MUČENYKY BUČA-IRPIN' \n\nNe snylos' poljanam j...


,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids
0,CLASSLA-web.mk.1000486,Forum,"Екшули, TCL ги прави смартфоновите, а TCL е см...","Ekshui, TCL makes smartphones, and TCL is hous...","CLASSLA-web.mk.1000486', 'domain': 'forum.carc...",Forum,"[▁Ек, шу, ли, ,, ▁T, CL, ▁ги, ▁прави, ▁смартфо...","[75430, 12213, 546, 4, 384, 37486, 1670, 10416..."
1,CLASSLA-web.mk.1009071,News,Red Valentino прогнозира бура од принтови за с...,Red Valentino predicts a storm of prints for n...,"CLASSLA-web.mk.1009071', 'domain': 'fashionel....",News,"[▁Red, ▁Valentino, ▁прогноз, ира, ▁бур, а, ▁од...","[6096, 166361, 45404, 6790, 21623, 59, 338, 44..."


,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids,text_latin
0,CLASSLA-web.mk.1000486,Forum,"Екшули, TCL ги прави смартфоновите, а TCL е см...","Ekshui, TCL makes smartphones, and TCL is hous...","CLASSLA-web.mk.1000486', 'domain': 'forum.carc...",Forum,"[▁Ек, шу, ли, ,, ▁T, CL, ▁ги, ▁прави, ▁смартфо...","[75430, 12213, 546, 4, 384, 37486, 1670, 10416...","Ekšuli, TCL gi pravi smartfonovite, a TCL e sm..."
1,CLASSLA-web.mk.1009071,News,Red Valentino прогнозира бура од принтови за с...,Red Valentino predicts a storm of prints for n...,"CLASSLA-web.mk.1009071', 'domain': 'fashionel....",News,"[▁Red, ▁Valentino, ▁прогноз, ира, ▁бур, а, ▁од...","[6096, 166361, 45404, 6790, 21623, 59, 338, 44...",Red Valentino prognozira bura od printovi za s...


In [ ]:
with pd.option_context('display.max_colwidth', 200):
	display(pd.DataFrame(main_dict["uk"]["dataset"])[["text", "text_latin"]].head(3))

In [ ]:
with pd.option_context('display.max_colwidth', 200):
	display(pd.DataFrame(main_dict["el"]["dataset"])[["text", "text_latin"]].head(3))

In [27]:
# Transliterate greek script to Latin, using the transliterate library: https://pypi.org/project/transliterate/
def transliterate_greek(text):
	from transliterate import translit, get_available_language_codes
	transl_text = translit(u"{}".format(text), 'el', reversed=True)
	return transl_text

for lang in list(main_dict.keys()):
	if lang == "el":
		# Open the df
		df = pd.DataFrame(main_dict[lang]["dataset"])
		display(df.head(2))
		# Transliterate and save text to new column
		df["text_latin"] = [transliterate_greek(x) for x in df["text"].to_list()]
		display(df.head(2))

		# Save the df
		main_dict[lang]["dataset"] = df.to_dict()

,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids
0,macocu.el.1525713,Instruction,Ενημέρωση του Pegasus Estiasi με τις εισερχόμε...,Update Pegasus Estiasi with Incoming Calls\n\n...,{'text_id': 'macocu.el.1525713'},Instruction,"[▁Ενημέρωση, ▁του, ▁Pegasus, ▁Esti, asi, ▁με, ...","[236422, 385, 241060, 60271, 1544, 558, 1713, ..."
1,macocu.el.3525724,Forum,Η τιμή της έκδοσης 8GB/ 128GB είναι 1.299 ευρώ...,"The price of 8GB/ 128GB is € 1,299, of the 12G...",{'text_id': 'macocu.el.3525724'},Forum,"[▁Η, ▁τιμή, ▁της, ▁έκδοση, ς, ▁8, GB, /, ▁128,...","[1700, 77118, 463, 110873, 235, 382, 8359, 64,..."


,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids,text_latin
0,macocu.el.1525713,Instruction,Ενημέρωση του Pegasus Estiasi με τις εισερχόμε...,Update Pegasus Estiasi with Incoming Calls\n\n...,{'text_id': 'macocu.el.1525713'},Instruction,"[▁Ενημέρωση, ▁του, ▁Pegasus, ▁Esti, asi, ▁με, ...","[236422, 385, 241060, 60271, 1544, 558, 1713, ...",Enimerosi toy Pegasus Estiasi me tis eiserchom...
1,macocu.el.3525724,Forum,Η τιμή της έκδοσης 8GB/ 128GB είναι 1.299 ευρώ...,"The price of 8GB/ 128GB is € 1,299, of the 12G...",{'text_id': 'macocu.el.3525724'},Forum,"[▁Η, ▁τιμή, ▁της, ▁έκδοση, ς, ▁8, GB, /, ▁128,...","[1700, 77118, 463, 110873, 235, 382, 8359, 64,...",i timi tis ekdosis 8GB/ 128GB einai 1.299 euro...


In [28]:
# Furthermore, let's also "normalize" the characters in all the languages to get rid of special characters, using the unidecode library: https://pypi.org/project/Unidecode/, and also lowercase all characters

def normalize(text):
	# First, make the text lowercase
	text = text.lower()
	from unidecode import unidecode
	norm_text = unidecode(u"{}".format(text))
	return norm_text

for lang in list(main_dict.keys()):
	# Open the df
	df = pd.DataFrame(main_dict[lang]["dataset"])

	if "lang" in ["uk", "mk", "el"]:
		# Take the transliterated text, normalize and save text to new column
		df["text_norm"] = [normalize(x) for x in df["text_latin"].to_list()]
		display(df.head(2))
	else:
		# Normalize and save text to new column
		df["text_norm"] = [normalize(x) for x in df["text"].to_list()]
		display(df.head(2))
	# Save the df
	main_dict[lang]["dataset"] = df.to_dict()

,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids,text_norm
0,macocu.mt.402244,News,"Angelo Chetcuti, se jkun qed jieħu post Bjorn ...","Angelo Chetcuti, will be replacing Bjorn Vassa...",{'text_id': 'macocu.mt.402244'},News,"[▁Angel, o, ▁Che, t, cuti, ,, ▁se, ▁j, kun, ▁q...","[26902, 31, 5024, 18, 64969, 4, 40, 1647, 6262...","angelo chetcuti, se jkun qed jiehu post bjorn ..."
1,macocu.mt.377203,Prose/Lyrical,Poltergeist jirreferi għal fenomeni oħra tal-m...,"Poltergeist refers to other woman's phenomena,...",{'text_id': 'macocu.mt.377203'},Opinion/Argumentation,"[▁Pol, ter, geist, ▁jir, re, feri, ▁g, ħ, al, ...","[9017, 720, 178490, 52826, 107, 26926, 706, 24...",poltergeist jirreferi ghal fenomeni ohra tal-m...


,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids,text_latin,text_norm
0,macocu.el.1525713,Instruction,Ενημέρωση του Pegasus Estiasi με τις εισερχόμε...,Update Pegasus Estiasi with Incoming Calls\n\n...,{'text_id': 'macocu.el.1525713'},Instruction,"[▁Ενημέρωση, ▁του, ▁Pegasus, ▁Esti, asi, ▁με, ...","[236422, 385, 241060, 60271, 1544, 558, 1713, ...",Enimerosi toy Pegasus Estiasi me tis eiserchom...,enemerose tou pegasus estiasi me tis eiserkhom...
1,macocu.el.3525724,Forum,Η τιμή της έκδοσης 8GB/ 128GB είναι 1.299 ευρώ...,"The price of 8GB/ 128GB is € 1,299, of the 12G...",{'text_id': 'macocu.el.3525724'},Forum,"[▁Η, ▁τιμή, ▁της, ▁έκδοση, ς, ▁8, GB, /, ▁128,...","[1700, 77118, 463, 110873, 235, 382, 8359, 64,...",i timi tis ekdosis 8GB/ 128GB einai 1.299 euro...,e time tes ekdoses 8gb/ 128gb einai 1.299 euro...


,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids,text_norm
0,macocu.tr.15851513,Instruction,AÖL Ders Seçimi ve Sınav Giriş Merkezi Belirle...,AÖL warned of frequent negligence and errors o...,{'text_id': 'macocu.tr.15851513'},Instruction,"[▁A, Ö, L, ▁Der, s, ▁Seçim, i, ▁ve, ▁Sınav, ▁G...","[62, 8655, 866, 1310, 7, 166134, 14, 173, 1762...",aol ders secimi ve sinav giris merkezi belirle...
1,macocu.tr.12699738,Legal,Banka promosyonu ihalesinde uygulanacak kriter...,Criteria to be applied in the tender for bank ...,{'text_id': 'macocu.tr.12699738'},Legal,"[▁Banka, ▁promo, syon, u, ▁i, hale, sinde, ▁uy...","[81847, 8891, 10270, 34, 17, 50742, 19209, 633...",banka promosyonu ihalesinde uygulanacak kriter...


,text_id,y_pred,text_length,domain,text,translation,metadata,y_true,tokens,token_ids,text_norm
0,macocu.sq.1061396,Opinion/Argumentation,341.0,fjalaejetes.org,Blog\n\n“Unë të kam dashur me një dashuri të p...,"Blog\n\n""I loved you with eternal love.""Jer 31...","{'text_id': 'macocu.sq.1061396', 'domain': 'fj...",Opinion/Argumentation,"[▁Blog, ▁“, U, në, ▁të, ▁kam, ▁dashur, ▁me, ▁n...","[5061, 52, 1062, 3208, 134, 3840, 57168, 163, ...","blog\n\n""une te kam dashur me nje dashuri te p..."
2,macocu.sq.183383,Legal,140.0,eukos.org,Liria nga keqtrajtimi\n\nKonventa e të Drejtav...,Freedom from mistreatment\n\nStudent Rights Co...,"{'text_id': 'macocu.sq.183383', 'domain': 'euk...",Legal,"[▁Li, ria, ▁nga, ▁keq, t, raj, timi, ▁Kon, ven...","[1261, 1651, 817, 39184, 18, 10185, 20520, 369...",liria nga keqtrajtimi\n\nkonventa e te drejtav...


,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids,text_norm
0,macocu.is.639516,Legal,[is] Því er við hæfi að reglur verði settar in...,[IS] It is therefore appropriate that rules be...,{'text_id': 'macocu.is.639516'},Legal,"[▁[, is, ], ▁Því, ▁er, ▁við, ▁hæ, fi, ▁að, ▁re...","[378, 164, 268, 139806, 72, 1497, 33423, 1029,...",[is] thvi er vid haefi ad reglur verdi settar ...
1,macocu.is.1301366,Instruction,Sækja um fulla aðild \n\nKennitala * \n\nNetfa...,Apply full membership\n\nSocial Security numbe...,{'text_id': 'macocu.is.1301366'},Information/Explanation,"[▁Sæ, kja, ▁um, ▁full, a, ▁að, ild, ▁Kenn, ita...","[71595, 28643, 286, 4393, 11, 389, 38472, 5906...",saekja um fulla adild \n\nkennitala * \n\nnetf...


,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids,text_latin,text_norm
0,macocu.uk.419381,Instruction,Нестандартний підхід для виготовлення Акваріум...,A non -standard approach for making aquarium.O...,{'text_id': 'macocu.uk.419381'},Instruction,"[▁Не, стандарт, ний, ▁підхід, ▁для, ▁виготовле...","[1087, 159257, 1394, 205827, 518, 166156, 1307...",Nestandartnyj pidxid dlja vyhotovlennja Akvari...,nestandartnii pidkhid dlia vigotovlennia akvar...
1,macocu.uk.16993168,Prose/Lyrical,МУЧЕНИКИ БУЧА-ІРПІНЬ \n\nНе снилось полянам й ...,The martyrs of Bucha-Irpin\n\nThe glades and t...,{'text_id': 'macocu.uk.16993168'},Prose/Lyrical,"[▁М, УЧ, ЕНИ, КИ, ▁, БУ, ЧА, -, ІР, П, ІН, Ь, ...","[1435, 87706, 78591, 38682, 6, 39932, 75333, 9...",MUČENYKY BUČA-IRPIN' \n\nNe snylos' poljanam j...,mucheniki bucha-irpin' \n\nne snilos' polianam...


,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids,text_norm
0,macocu.ca.2248072,Opinion/Argumentation,Pàgines \n\nEnfeinada \n\nPorto uns dies una m...,Pages\n\nCoined\n\nI have been a little busy f...,{'text_id': 'macocu.ca.2248072'},Forum,"[▁P, à, gine, s, ▁En, fei, nada, ▁Porto, ▁uns,...","[436, 1298, 63023, 7, 357, 51899, 28866, 24952...",pagines \n\nenfeinada \n\nporto uns dies una m...
1,macocu.ca.756254,Information/Explanation,Info \n\nLa Casa nova dels Banys de Sant Vicen...,Info\n\nThe Casa Nova dels Banys de Sant Vicen...,{'text_id': 'macocu.ca.756254'},Information/Explanation,"[▁Info, ▁La, ▁Casa, ▁nova, ▁dels, ▁Ban, ys, ▁d...","[14048, 239, 8591, 4678, 2323, 5458, 4778, 8, ...",info \n\nla casa nova dels banys de sant vicen...


,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids,text_latin,text_norm
0,CLASSLA-web.mk.1000486,Forum,"Екшули, TCL ги прави смартфоновите, а TCL е см...","Ekshui, TCL makes smartphones, and TCL is hous...","CLASSLA-web.mk.1000486', 'domain': 'forum.carc...",Forum,"[▁Ек, шу, ли, ,, ▁T, CL, ▁ги, ▁прави, ▁смартфо...","[75430, 12213, 546, 4, 384, 37486, 1670, 10416...","Ekšuli, TCL gi pravi smartfonovite, a TCL e sm...","ekshuli, tcl gi pravi smartfonovite, a tcl e s..."
1,CLASSLA-web.mk.1009071,News,Red Valentino прогнозира бура од принтови за с...,Red Valentino predicts a storm of prints for n...,"CLASSLA-web.mk.1009071', 'domain': 'fashionel....",News,"[▁Red, ▁Valentino, ▁прогноз, ира, ▁бур, а, ▁од...","[6096, 166361, 45404, 6790, 21623, 59, 338, 44...",Red Valentino prognozira bura od printovi za s...,red valentino prognozira bura od printovi za s...


,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids,text_norm
0,CLASSLA-web.hr.1033815,Promotion,"O proizvodu\nColor Transformer, za pametno i j...","About the Color Transformer product, for smart...","CLASSLA-web.hr.1033815', 'domain': 'hairshop.hr'}",Promotion,"[▁O, ▁proizvod, u, ▁Color, ▁Trans, former, ,, ...","[180, 43170, 34, 51193, 11062, 82772, 4, 80, 6...","o proizvodu\ncolor transformer, za pametno i j..."
2,CLASSLA-web.hr.1119579,Promotion,Sunčano selo / Sunny village\nNa obroncima Bil...,Sunshine / Sunny Village on the slopes of Bilo...,"CLASSLA-web.hr.1119579', 'domain': 'vikendi.com'}",Promotion,"[▁Sun, čan, o, ▁se, lo, ▁/, ▁Sunny, ▁village, ...","[7550, 17129, 31, 40, 365, 248, 151197, 54427,...",suncano selo / sunny village\nna obroncima bil...


,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids,text_norm
1,CLASSLA-web.sl.1087171,Opinion/Argumentation,Kitajsko mesto duhov\nV Notranji Mongoliji ras...,The Chinese ghost city in Inner Mongolia is gr...,"CLASSLA-web.sl.1087171', 'domain': 'mladina.si'}",News,"[▁Kita, jsko, ▁mesto, ▁duhov, ▁V, ▁Notranj, i,...","[12992, 30878, 22041, 87909, 310, 240213, 14, ...",kitajsko mesto duhov\nv notranji mongoliji ras...
2,CLASSLA-web.sl.1215246,Instruction,"Krompir skuhamo, olupimo in narežemo na tanke ...","Cook the potatoes, peel and cut into thin ring...","CLASSLA-web.sl.1215246', 'domain': 'emorje.com'}",Instruction,"[▁Krom, pir, ▁s, kuha, mo, ,, ▁olup, imo, ▁in,...","[128458, 21738, 91, 60863, 432, 4, 37663, 2414...","krompir skuhamo, olupimo in narezemo na tanke ..."


In [29]:
# Save the main dict
with open("manual-annotations/multilingual-genre-annotated-test-set.json", "w") as file:
	json.dump(main_dict, file)

In [12]:
# Open and normalize also the training dataset
# Open the tokenized df
train_df = pd.read_json("datasets/tokenized_datasets/X-GENRE-train-tokenized.json")
train_df.head(2)


,text,labels,dataset,language,tokens_train,token_ids,text_norm,tokens_train_norm
0,"Seeking All Things Brilliant ""I want people to...",Other,CORE,English,"[▁See, king, ▁All, ▁Things, ▁Br, illian, t, ▁""...","[6872, 6048, 3164, 119175, 13008, 162076, 18, ...","seeking all things brilliant ""i want people to...","[▁seeking, ▁all, ▁things, ▁brilliant, ▁"", i, ▁..."
1,Meet Orchid du Bois I first met Hayley Mowday ...,Other,CORE,English,"[▁Meet, ▁Or, ch, id, ▁du, ▁Bo, is, ▁I, ▁first,...","[72626, 3347, 206, 532, 115, 2460, 164, 87, 51...",meet orchid du bois i first met hayley mowday ...,"[▁meet, ▁or, ch, id, ▁du, ▁bois, ▁i, ▁first, ▁..."


In [34]:
# "normalize" the characters to get rid of special characters, using the unidecode library: https://pypi.org/project/Unidecode/, and also lowercase all characters

def normalize(text):
	# First, make the text lowercase
	text = text.lower()
	from unidecode import unidecode
	norm_text = unidecode(u"{}".format(text))
	return norm_text

# Normalize and save text to new column
train_df["text_norm"] = [normalize(x) for x in train_df["text"].to_list()]
display(train_df.head(2))

# Save the noramalized version
train_df.to_json("datasets/tokenized_datasets/X-GENRE-train-tokenized.json")

,text,labels,dataset,language,tokens_train,token_ids,text_norm
0,"Seeking All Things Brilliant ""I want people to...",Other,CORE,English,"[▁See, king, ▁All, ▁Things, ▁Br, illian, t, ▁""...","[6872, 6048, 3164, 119175, 13008, 162076, 18, ...","seeking all things brilliant ""i want people to..."
1,Meet Orchid du Bois I first met Hayley Mowday ...,Other,CORE,English,"[▁Meet, ▁Or, ch, id, ▁du, ▁Bo, is, ▁I, ▁first,...","[72626, 3347, 206, 532, 115, 2460, 164, 87, 51...",meet orchid du bois i first met hayley mowday ...


In [37]:
train_df[train_df["language"] == "Slovenian"].tail(2)

,text,labels,dataset,language,tokens_train,token_ids,text_norm
1140,"SPREJEM GASILSKEGA VOZILA GVM-1 <p/> V soboto,...",News,GINCO,Slovenian,"[▁S, PRE, JEM, ▁GA, SIL, SK, EGA, ▁V, OZ, ILA,...","[159, 94632, 164923, 23749, 68785, 10762, 5037...","sprejem gasilskega vozila gvm-1 <p/> v soboto,..."
1141,Dodano k projektu: Simple Shop Poročilo kreira...,Forum,GINCO,Slovenian,"[▁Doda, no, ▁k, ▁projektu, :, ▁Simple, ▁Shop, ...","[66987, 157, 472, 13181, 12, 60552, 24211, 663...",dodano k projektu: simple shop porocilo kreira...


## Overlap on dataset level

First, tokenize the training data

In [13]:
# To open as Pandas DataFrame:
train_df = pd.read_json("datasets/tokenized_datasets/X-GENRE-train-tokenized.json")

display(train_df.head(2))

,text,labels,dataset,language,tokens_train,token_ids,text_norm,tokens_train_norm
0,"Seeking All Things Brilliant ""I want people to...",Other,CORE,English,"[▁See, king, ▁All, ▁Things, ▁Br, illian, t, ▁""...","[6872, 6048, 3164, 119175, 13008, 162076, 18, ...","seeking all things brilliant ""i want people to...","[▁seeking, ▁all, ▁things, ▁brilliant, ▁"", i, ▁..."
1,Meet Orchid du Bois I first met Hayley Mowday ...,Other,CORE,English,"[▁Meet, ▁Or, ch, id, ▁du, ▁Bo, is, ▁I, ▁first,...","[72626, 3347, 206, 532, 115, 2460, 164, 87, 51...",meet orchid du bois i first met hayley mowday ...,"[▁meet, ▁or, ch, id, ▁du, ▁bois, ▁i, ▁first, ▁..."


In [4]:
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

# Tokenize the train dataset
tokens_train = []
token_list_train_norm = []

for text in tqdm(train_df["text_norm"].to_list()):
	encoded_text = tokenizer(text)
	# Take all tokens_train, except the beginning (<s>) and end (</s>) token
	current_tokens_train = tokenizer.convert_ids_to_tokens(encoded_text.input_ids)[1:-1]
	# Shorten the list to 512, as tokens after that were not observed by the classifier
	current_tokens_train = current_tokens_train[:512]
	tokens_train.append(current_tokens_train)
	token_list_train_norm.extend(current_tokens_train)

train_df["tokens_train_norm"] = tokens_train

train_df.head(3)

# Save the tokenized version
train_df.to_json("datasets/tokenized_datasets/X-GENRE-train-tokenized.json")

100%|██████████| 1772/1772 [00:09<00:00, 189.67it/s]


In [17]:
# Create a token list of all tokens in train
train_df = pd.read_json("datasets/tokenized_datasets/X-GENRE-train-tokenized.json")

token_list_train_norm = []

for i in train_df["tokens_train_norm"].to_list():
	token_list_train_norm.extend(i)

len(token_list_train_norm)


702133

Tokenize also the test sets

In [18]:
# Import the final dataset with test sets
with open("manual-annotations/multilingual-genre-annotated-test-set.json") as main_file:
	main_dict = json.load(main_file)

main_dict.keys()

dict_keys(['mt', 'el', 'tr', 'sq', 'is', 'uk', 'ca', 'mk', 'hr', 'sl'])

In [19]:
# Code with which the test sets were tokenized

# Define an array of token overlap
token_overlap_results = {}

# Loop through the datasets and calculate token overlap
for lang in list(main_dict.keys()):
	df = pd.DataFrame(main_dict[lang]["dataset"])

	tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

	tokens = []
	token_list = []

	print("Tokenizing text.")

	for text in tqdm(df["text_norm"].to_list()):
		encoded_text = tokenizer(text)
		# Take all tokens, except the beginning (<s>) and end (</s>) token
		current_tokens = tokenizer.convert_ids_to_tokens(encoded_text.input_ids)[1:-1]
		# Shorten the list to 512, as tokens after that were not observed by the classifier
		current_tokens = current_tokens[:512]
		tokens.append(current_tokens)
		token_list.extend(current_tokens)

	df["tokens_norm"] = tokens

	# Update the dataset in the dictionary
	main_dict[lang]["dataset"] = df.to_dict()

	# See how many tokens do not overlap
	no_overlap_counter = 0

	print("Calculating overlap.")

	# Save tokens that overlap for further inspection
	overlap_token_list = []

	# We calculate overlap by counting how many tokens do not appear in both sets
	for token in tqdm(token_list):
		if token not in token_list_train_norm:
			no_overlap_counter += 1
		else:
			overlap_token_list.append(token)

	# Out of all tokens in test set, how many do not overlap with train set?
	no_overlap_per = no_overlap_counter/len(token_list)

	# Calculate percentage of overlap based on that
	overlap_per = 1-no_overlap_per

	print(f"Number of tokens that overlap: {len(overlap_token_list)}")
	print(f"Number of different tokens that overlap: {len(set(overlap_token_list))}")
	print(f"Percentage of overlap: {overlap_per}")

	# Add the list of all tokens to the dictionary
	main_dict[lang]["token_overlap"]["overlap_percentage_norm"] = overlap_per
	main_dict[lang]["token_overlap"]["overlap_token_list_norm"] = overlap_token_list

	# Add to the results
	token_overlap_results[lang] = {"percentage": overlap_per, "overlap_list_size": len(overlap_token_list), "overlap_set_size": len(set(overlap_token_list))}


# Inspect the results
overlap_df = pd.DataFrame(token_overlap_results)

Tokenizing text.


100%|██████████| 80/80 [00:00<00:00, 394.94it/s]


Calculating overlap.


100%|██████████| 38674/38674 [00:50<00:00, 764.90it/s] 


Number of tokens that overlap: 35146
Number of different tokens that overlap: 3691
Percentage of overlap: 0.908775921807933
Tokenizing text.


...

Number of tokens that overlap: 23588
Number of different tokens that overlap: 3917
Percentage of overlap: 0.8738561849368355
Tokenizing text.


100%|██████████| 80/80 [00:00<00:00, 700.43it/s]


Calculating overlap.


100%|██████████| 26760/26760 [00:34<00:00, 774.76it/s]

Number of tokens that overlap: 26320
Number of different tokens that overlap: 4485
Percentage of overlap: 0.9835575485799701


In [20]:
overlap_df.transpose()

,percentage,overlap_list_size,overlap_set_size
mt,0.908776,35146.0,3691.0
el,0.895228,29231.0,2567.0
tr,0.729727,22623.0,2468.0
sq,0.779926,21175.0,2597.0
is,0.809069,24586.0,2447.0
uk,0.900191,31991.0,2957.0
ca,0.810781,22290.0,2666.0
mk,0.905123,27704.0,3293.0
hr,0.873856,23588.0,3917.0
sl,0.983558,26320.0,4485.0


In [21]:
print(overlap_df.transpose().sort_values(by="percentage", ascending=False).to_markdown())

|    |   percentage |   overlap_list_size |   overlap_set_size |
|:---|-------------:|--------------------:|-------------------:|
| sl |     0.983558 |               26320 |               4485 |
| mt |     0.908776 |               35146 |               3691 |
| mk |     0.905123 |               27704 |               3293 |
| uk |     0.900191 |               31991 |               2957 |
| el |     0.895228 |               29231 |               2567 |
| hr |     0.873856 |               23588 |               3917 |
| ca |     0.810781 |               22290 |               2666 |
| is |     0.809069 |               24586 |               2447 |
| sq |     0.779926 |               21175 |               2597 |
| tr |     0.729727 |               22623 |               2468 |


In [22]:
# Save the extended json dict
with open("manual-annotations/multilingual-genre-annotated-test-set.json", "w") as file:
	json.dump(main_dict, file)

## Label level

In [2]:
# Open the tokenized train df
train_df = pd.read_json("datasets/tokenized_datasets/X-GENRE-train-tokenized.json")
train_df.head(2)

,text,labels,dataset,language,tokens_train,token_ids,text_norm,tokens_train_norm
0,"Seeking All Things Brilliant ""I want people to...",Other,CORE,English,"[▁See, king, ▁All, ▁Things, ▁Br, illian, t, ▁""...","[6872, 6048, 3164, 119175, 13008, 162076, 18, ...","seeking all things brilliant ""i want people to...","[▁seeking, ▁all, ▁things, ▁brilliant, ▁"", i, ▁..."
1,Meet Orchid du Bois I first met Hayley Mowday ...,Other,CORE,English,"[▁Meet, ▁Or, ch, id, ▁du, ▁Bo, is, ▁I, ▁first,...","[72626, 3347, 206, 532, 115, 2460, 164, 87, 51...",meet orchid du bois i first met hayley mowday ...,"[▁meet, ▁or, ch, id, ▁du, ▁bois, ▁i, ▁first, ▁..."


In [ ]:
# Separate the train df into label-based dfs

# Create lists of tokens that are label based
label_tokens = {}
token_count = {}
type_count = {}

for label in ['Information/Explanation', 'News', 'Instruction','Opinion/Argumentation', 'Forum', 'Prose/Lyrical', 'Legal','Promotion']:
	print(f"Processing {label}")

	label_df = train_df[train_df["labels"] == label]
	display(label_df.head(3))

	# Create a list of tokens
	train_tokens_norm = []

	for i in label_df["tokens_train_norm"].to_list():
		train_tokens_norm.extend(i)

	print(f"Number of all tokens: {len(train_tokens_norm)}")

	# Add to dictionaries
	label_tokens[label] = train_tokens_norm
	token_count[label] = len(train_tokens_norm)
	type_count[label] = len(set(train_tokens_norm))

In [4]:
# Calculate numbers for each label
label_results_train_norm = pd.DataFrame({"token_count": token_count, "type_count": type_count})

print(label_results_train_norm.to_markdown())

|                         |   token_count |   type_count |
|:------------------------|--------------:|-------------:|
| Information/Explanation |        124518 |        11708 |
| News                    |        138026 |        11961 |
| Instruction             |         83634 |         7220 |
| Opinion/Argumentation   |        103447 |        10672 |
| Forum                   |         58952 |         7187 |
| Prose/Lyrical           |         46921 |         5119 |
| Legal                   |         28497 |         3707 |
| Promotion               |         88989 |         9768 |


In [5]:
# Save the label token count
with open("datasets/tokenized_datasets/X-GENRE-train-label-token-count-normalized-text.json", "w") as train_label_count_file:
	json.dump(label_tokens, train_label_count_file)

In [6]:
# Import the main dict for test sets
with open("manual-annotations/multilingual-genre-annotated-test-set.json", "r") as file:
	main_dict = json.load(file)

In [7]:
pd.DataFrame(main_dict["mt"]["dataset"]).head(2)

,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids,text_norm,tokens_norm
0,macocu.mt.402244,News,"Angelo Chetcuti, se jkun qed jieħu post Bjorn ...","Angelo Chetcuti, will be replacing Bjorn Vassa...",{'text_id': 'macocu.mt.402244'},News,"[▁Angel, o, ▁Che, t, cuti, ,, ▁se, ▁j, kun, ▁q...","[26902, 31, 5024, 18, 64969, 4, 40, 1647, 6262...","angelo chetcuti, se jkun qed jiehu post bjorn ...","[▁angel, o, ▁che, t, cuti, ,, ▁se, ▁j, kun, ▁q..."
1,macocu.mt.377203,Prose/Lyrical,Poltergeist jirreferi għal fenomeni oħra tal-m...,"Poltergeist refers to other woman's phenomena,...",{'text_id': 'macocu.mt.377203'},Opinion/Argumentation,"[▁Pol, ter, geist, ▁jir, re, feri, ▁g, ħ, al, ...","[9017, 720, 178490, 52826, 107, 26926, 706, 24...",poltergeist jirreferi ghal fenomeni ohra tal-m...,"[▁pol, ter, geist, ▁jir, re, feri, ▁, ghal, ▁f..."


In [ ]:
# Do the same as with the dataset, but on every language
lang_results = {}

for lang in list(main_dict.keys()):
	print(f"Processing {lang}")

	label_token_dict = {}

	# Current df
	df = pd.DataFrame(main_dict[lang]["dataset"])
	display(df.head(2))

	for label in ['Information/Explanation', 'News', 'Instruction','Opinion/Argumentation', 'Forum', 'Prose/Lyrical', 'Legal','Promotion']:
		print(f"Processing {label}")

		label_df = df[df["y_true"] == label]

		# Create a list of tokens
		token_list = []

		for i in label_df["tokens_norm"].to_list():
			token_list.extend(i)

		# Add to dictionary
		label_token_dict[label] = token_list

	# Add to main dict
	main_dict[lang]["token_overlap"]["label_level_token_lists_norm"] = label_token_dict

In [13]:
# Improved code to calculate token overlap - we can simply calculate which tokens do not overlap - meaning that they occur only in one of the dataset, and calculate the percentage overlap based on that
token_overlap_label_results = {}
results = {}

# Loop through the datasets and labels and calculate token overlap
for lang in list(main_dict.keys()):
	print(lang)
	# Create a dict for labels
	label_overlap = {}
	label_overlap_tokens = {}

	# loop through labels
	for label in ['Information/Explanation', 'News', 'Instruction','Opinion/Argumentation', 'Forum', 'Prose/Lyrical', 'Legal','Promotion']:
		print(label)
		token_list_test = main_dict[lang]["token_overlap"]["label_level_token_lists_norm"][label]

		token_list_train = label_tokens[label]

		# See how many tokens do not overlap
		no_overlap_counter = 0

		# Save tokens that overlap for further inspection
		overlap_token_list = []

		# We calculate overlap by counting how many tokens do not appear in both sets
		for token in tqdm(token_list_test):
			if token not in token_list_train:
				no_overlap_counter += 1
			else:
				overlap_token_list.append(token)

		# Out of all tokens in test set, how many do not overlap with train set?
		no_overlap_per = no_overlap_counter/len(token_list_test)

		# Calculate percentage of overlap based on that
		overlap_per = 1-no_overlap_per

		print(f"Percentage of overlap: {overlap_per}")

		label_overlap[label] = overlap_per
		label_overlap_tokens[label] = overlap_token_list

		# Add to the results
		results[f"{lang}-{label}"] = overlap_per

	# Add the list of all tokens to the dictionary
	main_dict[lang]["token_overlap"]["label_overlap_percentage_norm"] = label_overlap
	main_dict[lang]["token_overlap"]["label_overlap_token_list_norm"] = label_overlap_tokens

mt
Information/Explanation


100%|██████████| 6654/6654 [00:02<00:00, 2741.43it/s]


Percentage of overlap: 0.7790802524797115
News


100%|██████████| 6855/6855 [00:02<00:00, 2507.08it/s]


Percentage of overlap: 0.8023340627279358
Instruction


...

Percentage of overlap: 0.7410805300713558
Legal


100%|██████████| 3563/3563 [00:00<00:00, 13023.67it/s]


Percentage of overlap: 0.8004490597810834
Promotion


100%|██████████| 3298/3298 [00:00<00:00, 8642.27it/s]

Percentage of overlap: 0.9129775621588841


In [14]:
results

{'mt-Information/Explanation': 0.7790802524797115,
 'mt-News': 0.8023340627279358,
 'mt-Instruction': 0.6831737647188292,
 'mt-Opinion/Argumentation': 0.7689672050905532,
 'mt-Forum': 0.66796875,
 'mt-Prose/Lyrical': 0.68359375,
 'mt-Legal': 0.719921875,
 'mt-Promotion': 0.7187072503419973,
 'el-Information/Explanation': 0.7734947978850417,
 'el-News': 0.7590299277605779,
 'el-Instruction': 0.6099838969404187,
 'el-Opinion/Argumentation': 0.7217272104180945,
 'el-Forum': 0.6286532951289399,
 'el-Prose/Lyrical': 0.5938602681721947,
 'el-Legal': 0.4873096446700508,
 'el-Promotion': 0.6601612218922359,
 'tr-Information/Explanation': 0.6500198176773682,
 'tr-News': 0.6403141361256545,
 'tr-Instruction': 0.5206037374221371,
 'tr-Opinion/Argumentation': 0.597646645591851,
 'tr-Forum': 0.5817799158147925,
 'tr-Prose/Lyrical': 0.5465085638998682,
 'tr-Legal': 0.3967517401392111,
 'tr-Promotion': 0.5560538116591929,
 'sq-Information/Explanation': 0.6912599318955732,
 'sq-News': 0.66975806451612

In [15]:
# Inspect the results
overlap_df = pd.DataFrame({"label": list(results.keys()), "overlap": list(results.values())})
overlap_df

,label,overlap
0,mt-Information/Explanation,0.779080
1,mt-News,0.802334
2,mt-Instruction,0.683174
3,mt-Opinion/Argumentation,0.768967
4,mt-Forum,0.667969
...,...,...
75,sl-Opinion/Argumentation,0.923139
76,sl-Forum,0.841987
77,sl-Prose/Lyrical,0.741081
78,sl-Legal,0.800449


In [16]:
# Save the label-level-overlap
with open("datasets/label-level-token-overlap-normalized.csv", "w") as file:
	overlap_df.to_csv(file)

In [17]:
# Save the extended json dict
with open("manual-annotations/multilingual-genre-annotated-test-set.json", "w") as file:
	json.dump(main_dict, file)